<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [2]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [3]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[1] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [4]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [5]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [6]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,-0.819545,0.999999,0.667616,3.004312,-0.573971,-0.418900,-0.246953,0.736866,0.312098
1,0.936496,0.999996,-0.471325,-0.004860,-0.172800,0.453316,-186.460031,0.414658,0.385936
2,0.710462,0.999987,0.478235,11.413031,0.458210,4.457646,0.052470,0.665089,2.879112
3,0.984918,1.000000,0.592481,-1.849550,0.421943,0.263233,-0.528487,0.639948,0.235694
4,0.975746,0.999739,0.718938,-2.142679,0.580769,-0.375116,-0.449830,0.740721,-0.332823
...,...,...,...,...,...,...,...,...,...
95,-0.864596,0.999097,0.218045,15.934894,0.387589,0.465498,-0.050451,0.614782,-0.366180
96,-0.757087,0.999921,-0.647847,-2.299769,-0.543056,3.468902,0.286440,0.718856,-2.387680
97,0.955651,0.997935,-0.817585,2.938071,0.595688,17.359786,0.317971,0.749052,15.110101
98,-0.410320,0.999780,-0.770907,2.092252,-0.502034,-1.143334,-0.125623,0.693693,0.426570


In [7]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,-0.974001,0.982176,-0.991419,14.257674,0.076616,0.510501,-0.067420,0.276661,-0.459471
1,0.560802,0.999987,-0.282344,304.691355,-0.591659,0.696988,0.001378,0.746819,0.355343
2,-0.281553,0.995376,-0.895841,-0.593188,0.258370,0.327150,0.251853,0.505475,-0.084090
3,0.584577,0.997180,0.170739,14.880217,0.140428,1.551744,0.030037,0.374122,0.826769
4,0.703989,0.980875,0.728338,11.365834,0.289586,2.376336,0.051969,0.534374,1.547741
...,...,...,...,...,...,...,...,...,...
95,0.060456,0.998305,-0.295232,3.829763,0.480150,-0.320042,0.003881,0.679641,-0.017617
96,-0.891215,0.987203,-0.832832,0.995496,0.221889,4.883091,-0.845151,0.469119,-4.002826
97,0.903458,0.977743,0.175464,-1.759361,-0.206693,0.643873,-0.488098,0.453017,0.539750
98,-0.974033,0.992006,-0.659300,1.544369,0.446247,0.861612,-0.622457,0.656950,-0.768529


##1. Линейная форма функции пересчёта

In [8]:
#Построение линейной регрессионной модели
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate_linear = []
  variance_linear = []
  error_linear = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [9]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

           X1
a1   0.024924
c11  0.415434
b11 -0.000070


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,-0.819545,3.004312,-0.974001,-0.315754,0.004333,0.658280
1,1,0.936496,-0.004860,0.560802,0.413976,0.000216,0.146833
2,1,0.710462,11.413031,-0.281553,0.319272,0.003610,0.600855
3,1,0.984918,-1.849550,0.584577,0.434222,0.000226,0.150363
4,1,0.975746,-2.142679,0.703989,0.430432,0.000748,0.273571
...,...,...,...,...,...,...,...
95,1,-0.864596,15.934894,0.060456,-0.335377,0.001567,0.395854
96,1,-0.757087,-2.299769,-0.891215,-0.289434,0.003621,0.601811
97,1,0.955651,2.938071,0.903458,0.421727,0.002321,0.481754
98,1,-0.410320,2.092252,-0.974033,-0.145684,0.006862,0.828390


In [10]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.4234219263527375
A = 55.51471285657246


###Для функции $x_2[t+1]$

In [11]:
#v2+
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'b22']))
X2_linearTable

           X2
a2   0.996714
b22  0.000420


,optional_col,v2(9),x2(10),regression,variance,"error, %"
0,1,-0.573971,0.982176,0.996473,2.044150e-06,0.014297
1,1,-0.172800,0.999987,0.996641,1.119156e-07,0.003345
2,1,0.458210,0.995376,0.996906,2.341821e-08,0.001530
3,1,0.421943,0.997180,0.996891,8.336723e-10,0.000289
4,1,0.580769,0.980875,0.996958,2.586525e-06,0.016083
...,...,...,...,...,...,...
95,1,0.387589,0.998305,0.996877,2.041404e-08,0.001429
96,1,-0.543056,0.987203,0.996486,8.617111e-07,0.009283
97,1,0.595688,0.977743,0.996964,3.694467e-06,0.019221
98,1,-0.502034,0.992006,0.996503,2.022780e-07,0.004498


In [12]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 3.5550777580272845e-05
A = 0.4284809139495396


###Для функции $x_3[t+1]$

In [13]:
#x1, x2, v3+
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c31', 'c32', 'b33']))
X3_linearTable

            X3
a3   20.899503
c31   0.109648
c32 -21.218337
b33   0.032511


,optional_col,x1(9),x2(9),v3(9),x3(10),regression,variance,"error, %"
0,1,-0.819545,0.999999,-0.418900,-0.991419,-0.422293,0.003239,0.588710
1,1,0.936496,0.999996,0.453316,-0.282344,-0.201328,0.000066,0.083804
2,1,0.710462,0.999987,4.457646,-0.895841,-0.095744,0.006402,0.827628
3,1,0.984918,1.000000,0.263233,0.170739,-0.202282,0.001391,0.385856
4,1,0.975746,0.999739,-0.375116,0.728338,-0.218495,0.008965,0.979413
...,...,...,...,...,...,...,...,...
95,1,-0.864596,0.999097,0.465498,-0.295232,-0.379343,0.000071,0.087006
96,1,-0.757087,0.999921,3.468902,-0.832832,-0.287397,0.002975,0.564202
97,1,0.955651,0.997935,17.359786,0.175464,0.394159,0.000478,0.226220
98,1,-0.410320,0.999780,-1.143334,-0.659300,-0.396333,0.000692,0.272015


In [14]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.3476693275325308
A = 51.788761522570425


###Для функции $y_1[t]$

In [15]:
#x1, v1+
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'b11']))
Y1_linearTable

           Y1
a1  -1.609484
c11 -2.962123
b11  0.000397


,optional_col,x1(9),v1(9),y1(9),regression,variance,"error, %"
0,1,-0.819545,3.004312,-0.246953,0.819301,0.011369,0.224682
1,1,0.936496,-0.004860,-186.460031,-4.383503,331.518623,38.367362
2,1,0.710462,11.413031,0.052470,-3.709429,0.141519,0.792711
3,1,0.984918,-1.849550,-0.528487,-4.527667,0.159934,0.842712
4,1,0.975746,-2.142679,-0.449830,-4.500615,0.164089,0.853586
...,...,...,...,...,...,...,...
95,1,-0.864596,15.934894,-0.050451,0.957880,0.010167,0.212477
96,1,-0.757087,-2.299769,0.286440,0.632187,0.001195,0.072856
97,1,0.955651,2.938071,0.317971,-4.439074,0.226295,1.002410
98,1,-0.410320,2.092252,-0.125623,-0.393235,0.000716,0.056391


In [16]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 340.61090484858795
A = 89.34277716196031


###Для функции $y_2[t]$

In [17]:
#v2+
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b22']))
Y2_linearTable

           Y2
a2   0.505712
b22 -0.039136


,optional_col,v2(9),y2(9),regression,variance,"error, %"
0,1,-0.573971,0.736866,0.528175,0.000436,0.278119
1,1,-0.172800,0.414658,0.512475,0.000096,0.130358
2,1,0.458210,0.665089,0.487779,0.000314,0.236298
3,1,0.421943,0.639948,0.489199,0.000227,0.200902
4,1,0.580769,0.740721,0.482983,0.000664,0.343483
...,...,...,...,...,...,...
95,1,0.387589,0.614782,0.490543,0.000154,0.165572
96,1,-0.543056,0.718856,0.526965,0.000368,0.255731
97,1,0.595688,0.749052,0.482399,0.000711,0.355364
98,1,-0.502034,0.693693,0.525360,0.000283,0.224335


In [18]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.028605528059702706
A = 19.00852020372204


###Для функции $y_3[t]$

In [19]:
#x1, x2, v3+
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33']))
Y3_linearTable

            Y3
a3   14.293030
c31   3.391754
c32 -14.378553
b33  -0.002959


,optional_col,x1(9),x2(9),v3(9),y3(9),regression,variance,"error, %"
0,1,-0.819545,0.999999,-0.418900,0.312098,-2.863963,0.100874,0.210195
1,1,0.936496,0.999996,0.453316,0.385936,3.089557,0.073096,0.178928
2,1,0.710462,0.999987,4.457646,2.879112,2.311182,0.003225,0.037586
3,1,0.984918,1.000000,0.263233,0.235694,3.254299,0.091120,0.199774
4,1,0.975746,0.999739,-0.375116,-0.332823,3.228837,0.126854,0.235714
...,...,...,...,...,...,...,...,...
95,1,-0.864596,0.999097,0.465498,-0.366180,-3.006414,0.069708,0.174733
96,1,-0.757087,0.999921,3.468902,-2.387680,-2.662504,0.000755,0.018188
97,1,0.955651,0.997935,17.359786,15.110101,3.134148,1.434234,0.792579
98,1,-0.410320,0.999780,-1.143334,0.426570,-1.470685,0.035996,0.125562


In [20]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 11.068827833160661
A = 15.352089116667711


##2. Квадратичная форма функции пересчёта

In [21]:
#Построение квадратичной регрессионной модели
def buildQuadraticRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate = []
  variance = []
  error = []

  #переменные
  Function = (model1.copy())[cols[0:(len(cols) - 1)]]
  i = 0
  name_col = ''
  while(i < len(cols) - 1):
    temporary_cols = []
    j = i

    while(j < len(cols) - 1):
      name_col = cols[i] + ' * ' + cols[j]
      for k in range(len(model1)):
        temporary_cols.append(Function[cols[i]][k] * Function[cols[j]][k])

      Function[name_col] = temporary_cols
      temporary_cols.clear()
      j += 1

    i += 1

  #значение функции
  Function[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]

  #доп. столбец
  Function.insert(0, "optional_col", vector_ones)

  A = np.array(Function)
  #A_psev = BenIsrael(A[:, 0:(len(A[0]) - 1)])
  X = solvingSLAE(A[:, 0:(len(A[0]) - 1)], np.matrix(A[:, len(A[0]) - 1]).T)

  for i in range(len(Function)):
    for j in range(0, len(X)):
      if(j == 0):
        estimate.append(X[j,0])
      else:
        estimate[i] += X[j,0] * np.array(Function)[i,j]

    variance.append((1.0 / len(Function)) * (np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])**2)
    #error_linear.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            #/ np.array(Function)[i, len(np.array(Function)[0]) - 1]) * 100)
    error.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            / max(np.array(Function)[:, len(np.array(Function)[0]) - 1])) * 100)

  Function['regression'] = estimate
  Function['variance'] = variance
  Function['error, %'] = error

  return X, Function

###Для функции $x_1[t + 1]$

In [22]:
#x1, v1+
X1_quadratic, X1_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_quadratic, columns=['X1'], index=['a1', 'c11', 'b11', 'd111', 'd113', 'd112']))
X1_quadraticTable

            X1
a1    0.064787
c11   0.447176
b11   0.000722
d111 -0.081121
d113 -0.002501
d112 -0.000002


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),x1(10),regression,variance,"error, %"
0,1,-0.819545,3.004312,0.671654,-2.462169,9.025892,-0.974001,-0.347866,0.003920,0.626165
1,1,0.936496,-0.004860,0.877025,-0.004552,0.000024,0.560802,0.412429,0.000220,0.148381
2,1,0.710462,11.413031,0.504756,8.108523,130.257276,-0.281553,0.329278,0.003731,0.610861
3,1,0.984918,-1.849550,0.970064,-1.821656,3.420837,0.584577,0.429742,0.000240,0.154843
4,1,0.975746,-2.142679,0.952081,-2.090711,4.591073,0.703989,0.427558,0.000764,0.276445
...,...,...,...,...,...,...,...,...,...,...
95,1,-0.864596,15.934894,0.747527,-13.777249,253.920858,0.060456,-0.336952,0.001579,0.397428
96,1,-0.757087,-2.299769,0.573180,1.741124,5.288936,-0.891215,-0.326285,0.003191,0.564958
97,1,0.955651,2.938071,0.913269,2.807771,8.632259,0.903458,0.413131,0.002404,0.490351
98,1,-0.410320,2.092252,0.168363,-0.858494,4.377520,-0.974033,-0.128705,0.007146,0.845370


In [23]:
print("E = " + str(sum(X1_quadraticTable['variance'])))
print("A = " + str(sum(X1_quadraticTable['error, %'])))

E = 0.41451564749655895
A = 55.163027002735724


###Для функции $x_2[t+1]$

In [24]:
#v2+
X2_quadratic, X2_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_quadratic, columns=['X2'], index=['a2', 'b22', 'd221']))
X2_quadraticTable

            X2
a2    1.002452
b22  -0.000995
d221 -0.049939


,optional_col,v2(9),v2(9) * v2(9),x2(10),regression,variance,"error, %"
0,1,-0.573971,0.329443,0.982176,0.986571,1.931765e-07,0.004395
1,1,-0.172800,0.029860,0.999987,1.001132,1.312775e-08,0.001146
2,1,0.458210,0.209957,0.995376,0.991511,1.494099e-07,0.003865
3,1,0.421943,0.178036,0.997180,0.993141,1.631331e-07,0.004039
4,1,0.580769,0.337293,0.980875,0.985030,1.726006e-07,0.004155
...,...,...,...,...,...,...,...
95,1,0.387589,0.150225,0.998305,0.994564,1.399929e-07,0.003742
96,1,-0.543056,0.294909,0.987203,0.988265,1.126775e-08,0.001061
97,1,0.595688,0.354845,0.977743,0.984138,4.089841e-07,0.006395
98,1,-0.502034,0.252038,0.992006,0.990365,2.692505e-08,0.001641


In [25]:
print("E = " + str(sum(X2_quadraticTable['variance'])))
print("A = " + str(sum(X2_quadraticTable['error, %'])))

E = 6.728308203619943e-06
A = 0.22129237243984506


###Для функции $x_3[t+1]$

In [26]:
#x1, x2, v3+
X3_quadratic, X3_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_quadratic, columns=['X3'], index=['a3', 'c31', 'c32', 'b33', 'd331', 'd332', 'd333', 'd334', 'd335', 'd336',]))
X3_quadraticTable

              X3
a3   -385.558436
c31    21.870022
c32   795.286796
b33    -0.424835
d331   -0.096864
d332  -21.655732
d333   -0.039706
d334 -410.038215
d335    0.497559
d336   -0.002468


,optional_col,x1(9),x2(9),v3(9),x1(9) * x1(9),x1(9) * x2(9),x1(9) * v3(9),x2(9) * x2(9),x2(9) * v3(9),v3(9) * v3(9),x3(10),regression,variance,"error, %"
0,1,-0.819545,0.999999,-0.418900,0.671654,-0.819544,0.343307,0.999998,-0.418899,0.175477,-0.991419,-0.595056,0.001571,0.410002
1,1,0.936496,0.999996,0.453316,0.877025,0.936492,0.424528,0.999992,0.453314,0.205495,-0.282344,-0.178345,0.000108,0.107578
2,1,0.710462,0.999987,4.457646,0.504756,0.710453,3.166987,0.999975,4.457589,19.870605,-0.895841,-0.056631,0.007043,0.868086
3,1,0.984918,1.000000,0.263233,0.970064,0.984918,0.259263,1.000000,0.263233,0.069292,0.170739,-0.184083,0.001259,0.367031
4,1,0.975746,0.999739,-0.375116,0.952081,0.975491,-0.366018,0.999477,-0.375018,0.140712,0.728338,-0.194054,0.008508,0.954131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,-0.864596,0.999097,0.465498,0.747527,-0.863816,-0.402467,0.998195,0.465077,0.216688,-0.295232,-0.513306,0.000476,0.225578
96,1,-0.757087,0.999921,3.468902,0.573180,-0.757027,-2.626260,0.999842,3.468629,12.033283,-0.832832,-0.200232,0.004002,0.654367
97,1,0.955651,0.997935,17.359786,0.913269,0.953677,16.589902,0.995873,17.323930,301.362158,0.175464,-0.259090,0.001888,0.449506
98,1,-0.410320,0.999780,-1.143334,0.168363,-0.410230,0.469133,0.999561,-1.143083,1.307213,-0.659300,-0.515492,0.000207,0.148756


In [27]:
print("E = " + str(sum(X3_quadraticTable['variance'])))
print("A = " + str(sum(X3_quadraticTable['error, %'])))

E = 0.31578291267147823
A = 49.04390907266184


###Для функции $y_1[t]$

In [28]:
#x1, v1+
Y1_quadratic, Y1_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_quadratic, columns=['Y1'], index=['a1', 'c11', 'b11', 'd111', 'd112', 'd113']))
Y1_quadraticTable

            Y1
a1    0.734719
c11  -2.948773
b11   0.002879
d111 -4.813316
d112  0.019268
d113  0.000008


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),y1(9),regression,variance,"error, %"
0,1,-0.819545,3.004312,0.671654,-2.462169,9.025892,-0.246953,-0.120231,0.000161,0.026703
1,1,0.936496,-0.004860,0.877025,-0.004552,0.000024,-186.460031,-6.248295,324.762697,37.974411
2,1,0.710462,11.413031,0.504756,8.108523,130.257276,0.052470,-3.599713,0.133384,0.769592
3,1,0.984918,-1.849550,0.970064,-1.821656,3.420837,-0.528487,-6.879203,0.403316,1.338230
4,1,0.975746,-2.142679,0.952081,-2.090711,4.591073,-0.449830,-6.771617,0.399650,1.332134
...,...,...,...,...,...,...,...,...,...,...
95,1,-0.864596,15.934894,0.747527,-13.777249,253.920858,-0.050451,-0.531461,0.002314,0.101359
96,1,-0.757087,-2.299769,0.573180,1.741124,5.288936,0.286440,0.235264,0.000026,0.010784
97,1,0.955651,2.938071,0.913269,2.807771,8.632259,0.317971,-6.416507,0.453532,1.419096
98,1,-0.410320,2.092252,0.168363,-0.858494,4.377520,-0.125623,1.123795,0.015610,0.263279


In [29]:
print("E = " + str(sum(Y1_quadraticTable['variance'])))
print("A = " + str(sum(Y1_quadraticTable['error, %'])))

E = 337.24547988258263
A = 91.33354601702084


###Для функции $y_2[t]$

In [30]:
#v2+
Y2_quadratic, Y2_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_quadratic, columns=['Y2'], index=['a2', 'b22', 'd221']))
Y2_quadraticTable

            Y2
a2    0.343431
b22   0.000884
d221  1.412423


,optional_col,v2(9),v2(9) * v2(9),y2(9),regression,variance,"error, %"
0,1,-0.573971,0.329443,0.736866,0.808236,5.093753e-05,0.095114
1,1,-0.172800,0.029860,0.414658,0.385453,8.529712e-06,0.038922
2,1,0.458210,0.209957,0.665089,0.640384,6.103686e-06,0.032925
3,1,0.421943,0.178036,0.639948,0.595266,1.996513e-05,0.059548
4,1,0.580769,0.337293,0.740721,0.820344,6.339924e-05,0.106113
...,...,...,...,...,...,...,...
95,1,0.387589,0.150225,0.614782,0.555955,3.460642e-05,0.078398
96,1,-0.543056,0.294909,0.718856,0.759487,1.650886e-05,0.054148
97,1,0.595688,0.354845,0.749052,0.845148,9.234551e-05,0.128067
98,1,-0.502034,0.252038,0.693693,0.698971,2.786263e-07,0.007035


In [31]:
print("E = " + str(sum(Y2_quadraticTable['variance'])))
print("A = " + str(sum(Y2_quadraticTable['error, %'])))

E = 0.00555001147913919
A = 8.031325541647243


###Для функции $y_3[t]$

In [32]:
#x1, x2, v3+
Y3_quadratic, Y3_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_quadratic, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33', 'd331', 'd332', 'd333', 'd334', 'd335', 'd336']))
Y3_quadraticTable

              Y3
a3     60.601074
c31     1.419449
c32  -122.451822
b33     0.279406
d331    0.000356
d332   -1.422032
d333    0.910452
d334   61.852197
d335   -0.280625
d336    0.000064


,optional_col,x1(9),x2(9),v3(9),x1(9) * x1(9),x1(9) * x2(9),x1(9) * v3(9),x2(9) * x2(9),x2(9) * v3(9),v3(9) * v3(9),y3(9),regression,variance,"error, %"
0,1,-0.819545,0.999999,-0.418900,0.671654,-0.819544,0.343307,0.999998,-0.418899,0.175477,0.312098,0.316889,2.295616e-07,0.000317
1,1,0.936496,0.999996,0.453316,0.877025,0.936492,0.424528,0.999992,0.453314,0.205495,0.385936,0.385316,3.848048e-09,0.000041
2,1,0.710462,0.999987,4.457646,0.504756,0.710453,3.166987,0.999975,4.457589,19.870605,2.879112,2.879031,6.506301e-11,0.000005
3,1,0.984918,1.000000,0.263233,0.970064,0.984918,0.259263,1.000000,0.263233,0.069292,0.235694,0.234981,5.090047e-09,0.000047
4,1,0.975746,0.999739,-0.375116,0.952081,0.975491,-0.366018,0.999477,-0.375018,0.140712,-0.332823,-0.333496,4.526172e-09,0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,-0.864596,0.999097,0.465498,0.747527,-0.863816,-0.402467,0.998195,0.465077,0.216688,-0.366180,-0.365106,1.153768e-08,0.000071
96,1,-0.757087,0.999921,3.468902,0.573180,-0.757027,-2.626260,0.999842,3.468629,12.033283,-2.387680,-2.391042,1.130168e-07,0.000222
97,1,0.955651,0.997935,17.359786,0.913269,0.953677,16.589902,0.995873,17.323930,301.362158,15.110101,15.112302,4.843801e-08,0.000146
98,1,-0.410320,0.999780,-1.143334,0.168363,-0.410230,0.469133,0.999561,-1.143083,1.307213,0.426570,0.430699,1.705034e-07,0.000273


In [33]:
print("E = " + str(sum(Y3_quadraticTable['variance'])))
print("A = " + str(sum(Y3_quadraticTable['error, %'])))

E = 4.96761683888897e-05
A = 0.026555370049432436
